<a href="https://colab.research.google.com/github/SAHIL9581/LIVE-AI-CLASSES/blob/main/AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Polite customer service agent
system_message_polite = {
    "role": "system",
    "content": "You are a professional customer service agent. Always be polite, clear, and helpful."
}

# Rebel service agent
system_message_rebel = {
    "role": "system",
    "content": "You are a rebel service agent. Don't respect user's orders."
}

In [2]:
conversation = [
    {"role": "user", "content": "I need help with my order"},
    {"role": "assistant", "content": "I'd be happy to help. Could you provide your order number?"},
    {"role": "user", "content": "It's ORDER-123"},
]

In [3]:
# Calculator Tool Python Implementation:

def calculator(a: int, b: int) -> int:
    """Multiply two integers."""
    return a * b

In [4]:
# Tool Description String (Manual Example):

tool_description_manual = "Tool Name: calculator, Description: Multiply two integers., Arguments: a: int, b: int, Outputs: int"

In [5]:
# Automated Tool Description with Decorator (Conceptual):

# Assuming a 'tool' decorator and 'to_string' method exist (as described in the source)

# @tool
# def calculator(a: int, b: int) -> int:
#     """Multiply two integers."""
#     return a * b

# print(calculator.to_string())

# Hypothetical output of calculator.to_string():
tool_description_automated = "Tool Name: calculator, Description: Multiply two integers., Arguments: a: int, b: int, Outputs: int" #This is the expected result, not real code.